# Задание 5. Предсказание стоимости квартир.

В некоем несуществующем городе N цены на недвижимость ведут себя очень непредска-
зуемо. Чтобы научиться их прогнозировать, не обойтись без помощи ИИ. Однако у нас

есть база данных сервиса «Объявления ВКонтакте», и по текстам из этих объявлений
можно выявить полезные закономерности, влияющие на цену.

Перед вами стоит задача разработать алгоритм машинного обучения, предсказывающий цену недвижимости в городе N по данным из объявления о продаже.

Метрика оценки качества решения — RMSE.

[Соревнование](https://www.kaggle.com/competitions/task-5-demo/overview) на kaggle

[Демоверсия](https://drive.google.com/file/d/1fI_X1dEmqRASVtYwLfOPwfFuZyqf1-3n/view?usp=sharing) заключительного этапа

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [ ]:
train

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-1,560,NaN,2,59,3,0,30.0,1.0,...,0,0,0,0,0,0,0,0,0,4510000
1,4,2011-1,232,0.0,3,60,3,0,25.0,NaN,...,0,0,0,0,0,0,0,0,0,3335000
2,5,2011-1,453,1.0,5,52,2,0,30.0,1.0,...,0,0,0,0,0,0,0,0,0,3612000
3,6,2011-1,44,NaN,1,33,1,0,30.0,1.0,...,0,0,0,0,0,0,0,0,0,559000
4,7,2011-1,74,2.0,16,44,1,2,20.0,0.0,...,0,0,0,0,0,0,0,0,0,4080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,85768,2012-12,379,0.0,8,33,1,0,30.0,NaN,...,0,0,0,0,0,0,0,0,0,4535000
59996,85769,2012-12,476,1.0,9,35,1,1,30.0,0.0,...,0,0,0,0,0,0,0,0,0,5175000
59997,85771,2012-12,261,NaN,4,154,4,1,20.0,NaN,...,0,0,0,0,0,0,0,0,0,29187000
59998,85772,2012-12,20,1.0,2,62,3,1,15.0,0.0,...,0,0,0,0,0,0,0,0,0,5209000


В датасете имеются пропуски и категориальные признаки.

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          60000 non-null  int64  
 1   date        60000 non-null  object 
 2   street_id   60000 non-null  int64  
 3   build_tech  42041 non-null  float64
 4   floor       60000 non-null  int64  
 5   area        60000 non-null  int64  
 6   rooms       60000 non-null  int64  
 7   balcon      60000 non-null  int64  
 8   metro_dist  57042 non-null  float64
 9   g_lift      41957 non-null  float64
 10  n_photos    60000 non-null  int64  
 11  kw1         60000 non-null  int64  
 12  kw2         60000 non-null  int64  
 13  kw3         60000 non-null  int64  
 14  kw4         60000 non-null  int64  
 15  kw5         60000 non-null  int64  
 16  kw6         60000 non-null  int64  
 17  kw7         60000 non-null  int64  
 18  kw8         60000 non-null  int64  
 19  kw9         60000 non-nul

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          10000 non-null  int64  
 1   date        10000 non-null  object 
 2   street_id   10000 non-null  int64  
 3   build_tech  7080 non-null   float64
 4   floor       10000 non-null  int64  
 5   area        10000 non-null  int64  
 6   rooms       10000 non-null  int64  
 7   balcon      10000 non-null  int64  
 8   metro_dist  9508 non-null   float64
 9   g_lift      6967 non-null   float64
 10  n_photos    10000 non-null  int64  
 11  kw1         10000 non-null  int64  
 12  kw2         10000 non-null  int64  
 13  kw3         10000 non-null  int64  
 14  kw4         10000 non-null  int64  
 15  kw5         10000 non-null  int64  
 16  kw6         10000 non-null  int64  
 17  kw7         10000 non-null  int64  
 18  kw8         10000 non-null  int64  
 19  kw9         10000 non-null

In [ ]:
train.isna().sum()

id                0
date              0
street_id         0
build_tech    17959
floor             0
area              0
rooms             0
balcon            0
metro_dist     2958
g_lift        18043
n_photos          0
kw1               0
kw2               0
kw3               0
kw4               0
kw5               0
kw6               0
kw7               0
kw8               0
kw9               0
kw10              0
kw11              0
kw12              0
kw13              0
price             0
dtype: int64

In [ ]:
test.isna().sum()

id               0
date             0
street_id        0
build_tech    2920
floor            0
area             0
rooms            0
balcon           0
metro_dist     492
g_lift        3033
n_photos         0
kw1              0
kw2              0
kw3              0
kw4              0
kw5              0
kw6              0
kw7              0
kw8              0
kw9              0
kw10             0
kw11             0
kw12             0
kw13             0
dtype: int64

Удалим стобец "id"

In [ ]:
ids=test['id']
test=test.drop(columns=['id'])
train=train.drop(columns=['id'])

Представим время в формате unix-time

In [ ]:
train['date'] = pd.to_datetime(train['date'], format='%Y-%m').astype(int)/10**9
test['date'] = pd.to_datetime(test['date'], format='%Y-%m').astype(int)/10**9

<ipython-input-175-fbd263592533>:1: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  train['date'] = pd.to_datetime(train['date'], format='%Y-%m').astype(int)/10**9
<ipython-input-175-fbd263592533>:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  test['date'] = pd.to_datetime(test['date'], format='%Y-%m').astype(int)/10**9


In [ ]:
train

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,1.293840e+09,560,NaN,2,59,3,0,30.0,1.0,5,...,0,0,0,0,0,0,0,0,0,4510000
1,1.293840e+09,232,0.0,3,60,3,0,25.0,NaN,3,...,0,0,0,0,0,0,0,0,0,3335000
2,1.293840e+09,453,1.0,5,52,2,0,30.0,1.0,1,...,0,0,0,0,0,0,0,0,0,3612000
3,1.293840e+09,44,NaN,1,33,1,0,30.0,1.0,0,...,0,0,0,0,0,0,0,0,0,559000
4,1.293840e+09,74,2.0,16,44,1,2,20.0,0.0,1,...,0,0,0,0,0,0,0,0,0,4080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1.354320e+09,379,0.0,8,33,1,0,30.0,NaN,0,...,0,0,0,0,0,0,0,0,0,4535000
59996,1.354320e+09,476,1.0,9,35,1,1,30.0,0.0,2,...,0,0,0,0,0,0,0,0,0,5175000
59997,1.354320e+09,261,NaN,4,154,4,1,20.0,NaN,7,...,0,0,0,0,0,0,0,0,0,29187000
59998,1.354320e+09,20,1.0,2,62,3,1,15.0,0.0,0,...,0,0,0,0,0,0,0,0,0,5209000


**Матрица корреляции**

In [ ]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
date,1.000000,0.000325,-0.001303,0.008818,-0.000274,-0.000992,0.002394,0.001177,-0.000120,0.001259,0.000182,0.006295,-0.001658,0.009329,-0.005570,-0.001908,-0.006854,-0.010825,0.002251,0.000138,-0.007245,0.002876,0.008793,0.080384
street_id,0.000325,1.000000,-0.014384,-0.010308,-0.009874,0.005370,-0.009691,0.007232,-0.007224,0.006456,-0.004307,0.002703,0.003312,0.005435,0.003062,-0.006876,-0.001276,0.002955,0.008956,-0.007540,0.005089,-0.003114,-0.000480,-0.020210
build_tech,-0.001303,-0.014384,1.000000,0.180996,0.321452,0.122362,0.785588,0.015723,0.002464,0.025753,0.002479,-0.027615,-0.079090,-0.073722,-0.054809,-0.037659,-0.026618,0.076167,0.071789,0.040362,0.014394,0.089183,0.030573,0.367091
floor,0.008818,-0.010308,0.180996,1.000000,0.149832,-0.002883,0.144775,0.008319,-0.004644,0.005907,0.001977,-0.037596,-0.048501,-0.005731,0.022204,0.008740,0.070536,-0.055043,0.025409,0.015088,0.009924,0.065424,0.004128,0.202357
area,-0.000274,-0.009874,0.321452,0.149832,1.000000,0.790802,0.253124,0.002867,-0.001942,0.169154,-0.001701,-0.065745,-0.041098,-0.018800,-0.012798,0.012808,0.008645,0.001288,-0.003486,0.094349,0.054515,0.021249,0.040137,0.665169
rooms,-0.000992,0.005370,0.122362,-0.002883,0.790802,1.000000,0.097325,0.006061,0.000012,0.212489,0.000071,-0.008275,-0.011657,-0.008352,-0.002930,-0.002115,-0.002904,0.015191,0.006856,0.025572,0.019397,-0.007283,0.009341,0.375341
balcon,0.002394,-0.009691,0.785588,0.144775,0.253124,0.097325,1.000000,0.010557,0.011170,0.017678,0.004783,-0.023814,-0.061316,-0.057599,-0.045554,-0.028655,-0.021509,0.055560,0.061443,0.032120,0.010110,0.068023,0.028473,0.286084
metro_dist,0.001177,0.007232,0.015723,0.008319,0.002867,0.006061,0.010557,1.000000,-0.001424,0.005962,0.000698,-0.004587,-0.005581,0.003006,0.000640,0.000520,-0.006690,0.001605,0.006578,-0.003785,0.001073,0.001965,-0.001569,-0.095750
g_lift,-0.000120,-0.007224,0.002464,-0.004644,-0.001942,0.000012,0.011170,-0.001424,1.000000,-0.000356,-0.000703,-0.005985,-0.003356,-0.002146,-0.001947,-0.007458,0.004745,0.001572,-0.003246,-0.002075,-0.002824,-0.006050,-0.001849,-0.004084
n_photos,0.001259,0.006456,0.025753,0.005907,0.169154,0.212489,0.017678,0.005962,-0.000356,1.000000,0.037585,-0.013487,-0.001774,0.006742,0.005318,0.005320,0.006618,0.016161,0.013265,-0.002610,0.007298,0.005434,0.002622,0.185345


Удалим столбец g_lift, имеющий большое количество пропусков (около 30%) и малую по модулю корреляцию с целевой переменной

In [ ]:
train=train.drop(columns=['g_lift'])
test=test.drop(columns=['g_lift'])


In [ ]:
train

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,n_photos,kw1,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,1.293840e+09,560,NaN,2,59,3,0,30.0,5,0,...,0,0,0,0,0,0,0,0,0,4510000
1,1.293840e+09,232,0.0,3,60,3,0,25.0,3,0,...,0,0,0,0,0,0,0,0,0,3335000
2,1.293840e+09,453,1.0,5,52,2,0,30.0,1,0,...,0,0,0,0,0,0,0,0,0,3612000
3,1.293840e+09,44,NaN,1,33,1,0,30.0,0,0,...,0,0,0,0,0,0,0,0,0,559000
4,1.293840e+09,74,2.0,16,44,1,2,20.0,1,0,...,0,0,0,0,0,0,0,0,0,4080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1.354320e+09,379,0.0,8,33,1,0,30.0,0,0,...,0,0,0,0,0,0,0,0,0,4535000
59996,1.354320e+09,476,1.0,9,35,1,1,30.0,2,0,...,0,0,0,0,0,0,0,0,0,5175000
59997,1.354320e+09,261,NaN,4,154,4,1,20.0,7,0,...,0,0,0,0,0,0,0,0,0,29187000
59998,1.354320e+09,20,1.0,2,62,3,1,15.0,0,0,...,0,0,0,0,0,0,0,0,0,5209000


Заполним пропуски значениями, наиболее часто встречающимися в соответствующем столбце

In [ ]:
train['build_tech'].fillna(train['build_tech'].mode()[0], inplace=True)
test['build_tech'].fillna(test['build_tech'].mode()[0], inplace=True)
train['metro_dist'].fillna(train['metro_dist'].mode()[0], inplace=True)
test['metro_dist'].fillna(test['metro_dist'].mode()[0], inplace=True)

Замена выбросов на nan

In [ ]:
hd=train.head()
for j in hd:
  for x in [j]:
      q75,q25 = np.percentile(train.loc[:,x],[75,25])
      intr_qr = q75-q25

      max = q75+(1.5*intr_qr)
      min = q25-(1.5*intr_qr)

      train.loc[train[x] < min,x] = np.nan
      train.loc[train[x] > max,x] = np.nan

In [ ]:
train.isna().sum()

date             0
street_id        0
build_tech       0
floor         1441
area          1639
rooms            0
balcon           0
metro_dist       0
n_photos        67
kw1           3319
kw2              0
kw3            428
kw4            378
kw5            237
kw6             94
kw7             53
kw8            494
kw9            464
kw10           134
kw11            20
kw12            72
kw13            12
price         3454
dtype: int64

Удаление образовавшихся пропусков

In [ ]:
hd=train.head()
for j in hd:
  train = train.dropna(axis='index', how='any', subset=[j])

In [ ]:
train

,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,n_photos,kw1,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,1.293840e+09,560,0.0,2,59,3,0,30.0,5,0,...,0,0,0,0,0,0,0,0,0,4510000
1,1.293840e+09,232,0.0,3,60,3,0,25.0,3,0,...,0,0,0,0,0,0,0,0,0,3335000
2,1.293840e+09,453,1.0,5,52,2,0,30.0,1,0,...,0,0,0,0,0,0,0,0,0,3612000
3,1.293840e+09,44,0.0,1,33,1,0,30.0,0,0,...,0,0,0,0,0,0,0,0,0,559000
4,1.293840e+09,74,2.0,16,44,1,2,20.0,1,0,...,0,0,0,0,0,0,0,0,0,4080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1.354320e+09,379,0.0,8,33,1,0,30.0,0,0,...,0,0,0,0,0,0,0,0,0,4535000
59996,1.354320e+09,476,1.0,9,35,1,1,30.0,2,0,...,0,0,0,0,0,0,0,0,0,5175000
59997,1.354320e+09,261,0.0,4,154,4,1,20.0,7,0,...,0,0,0,0,0,0,0,0,0,29187000
59998,1.354320e+09,20,1.0,2,62,3,1,15.0,0,0,...,0,0,0,0,0,0,0,0,0,5209000


Преобразование категориальных признаков. Для каждого возможного значения из столбцов build_tech, floor, rooms, street_id создаётся новый столбец со значением 0 или 1 (если значение соответсвующее данному столбцу равно значению, которое было в исходном столбце). Происходит значительное увеличение количества столбцов в датасете

In [ ]:
c=train.copy()
c=c.append(test)
c=  pd.get_dummies(c, columns=['build_tech', 'floor', 'rooms', 'street_id'])
size=len(train)
train=c[:size]
test=c[size:]

In [ ]:
train

,date,area,balcon,metro_dist,n_photos,kw1,kw2,kw3,kw4,kw5,...,street_id_662,street_id_663,street_id_664,street_id_665,street_id_666,street_id_667,street_id_668,street_id_669,street_id_670,street_id_671
0,1.293840e+09,59,0,30.0,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.293840e+09,60,0,25.0,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.293840e+09,52,0,30.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.293840e+09,33,0,30.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.293840e+09,44,2,20.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,1.354320e+09,33,0,30.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1.354320e+09,35,1,30.0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,1.354320e+09,154,1,20.0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,1.354320e+09,62,1,15.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Разделение данных на матрицу признаков и столбец целевой переменной

In [ ]:
X = train.drop(columns=['price'])
y = train['price']

Разделение исходной выборки на test и train

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f'Train dataset size: {X_train.shape}, {y_train.shape}')
print(f'Test dataset size: {X_test.shape}, {y_test.shape}')

Train dataset size: (45000, 724), (45000,)
Test dataset size: (15000, 724), (15000,)


Решение на основе линейного дискриминантного анализа

In [ ]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model=LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

LinearDiscriminantAnalysis()

Воспользуемся описанной в демоверсии идеей решения и применим градиентный бустинг

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.4 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
import sklearn
model=CatBoostRegressor(loss_function='RMSE', iterations=3000)
model.fit(X_train, y_train)

Learning rate set to 0.030584
0:	learn: 4120377.2583856	total: 9.47ms	remaining: 28.4s
1:	learn: 4063603.1688350	total: 18.7ms	remaining: 28.1s
2:	learn: 4006706.9844434	total: 28.2ms	remaining: 28.2s
3:	learn: 3953840.2473545	total: 37.4ms	remaining: 28s
4:	learn: 3901468.2089857	total: 47ms	remaining: 28.2s
5:	learn: 3851646.3021164	total: 56.8ms	remaining: 28.3s
6:	learn: 3804063.2195552	total: 71.8ms	remaining: 30.7s
7:	learn: 3759640.7156847	total: 81.6ms	remaining: 30.5s
8:	learn: 3717454.9904039	total: 91.3ms	remaining: 30.3s
9:	learn: 3675243.2531191	total: 102ms	remaining: 30.4s
10:	learn: 3636828.5475939	total: 111ms	remaining: 30.1s
11:	learn: 3600701.7773928	total: 120ms	remaining: 29.9s
12:	learn: 3563848.9743239	total: 129ms	remaining: 29.7s
13:	learn: 3529840.3763753	total: 138ms	remaining: 29.4s
14:	learn: 3495921.0129475	total: 149ms	remaining: 29.7s
15:	learn: 3464388.1143200	total: 158ms	remaining: 29.5s
16:	learn: 3435113.4517637	total: 168ms	remaining: 29.4s
17:	le

Оценка полученной точности на тестовых данных

In [ ]:
y_pred=model.predict(X_test)
sklearn.metrics.mean_squared_error(y_true=y_test, y_pred=y_pred, squared=False)

1570718.571865081

Создаём файл с результатом для тестовых данных

In [ ]:
result=model.predict(test)
from google.colab import files

print(type(ids))
ids.to_frame(name='id')
result=pd.Series(result).to_frame(name='price')

ans = pd.concat([ids, result], axis= 1 )
t=ans.to_csv('prediction.csv', header=['id', 'price'], index=False)
files.download('prediction.csv')

<class 'pandas.core.series.Series'>


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>